In [157]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from transformers import BitsAndBytesConfig
# 원하는 GPU를 지정 (예: 두 번째 GPU 사용)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda:0


### 데이터셋

In [153]:
import pandas as pd
import pyarrow as pa
from datasets import Dataset, DatasetDict, Value, Features

# 1. CSV 파일 불러오기
df = pd.read_csv("extracted_documents.csv")
df = df.fillna("")

# 2. 필요한 컬럼만 추출하고 이름 변경
df['input_ids'] = df['original_text']
df['labels']  = df['summary_text']
df = df[['input_ids', 'labels']]

# 3. 문자열 길이 확인 (디버깅용)
print(f"최대 input_ids 길이: {df['input_ids'].str.len().max()}")
print(f"최대 labels 길이: {df['labels'].str.len().max()}")

# 4. PyArrow Table로 변환 (large_string 적용)
schema = pa.schema([
    ('input_ids', pa.large_string()),
    ('labels', pa.large_string())
])
table = pa.Table.from_pandas(df, schema=schema)



# 5. Dataset으로 변환
dataset = Dataset.from_dict({col: table[col].to_pandas() for col in table.schema.names})

# 6. train/validation/test 데이터셋 분할 (8:1:1 비율 예시)
#   - train_test_split() 함수를 두 번 호출해서
#     먼저 train: 80%, test: 20% 분리하고,
#     그 뒤 test 부분을 다시 50%씩 나눠 validation: 10%, test: 10% 로 분할
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
temp_dataset  = split_dataset['test'].train_test_split(test_size=0.5, seed=42)

final_dataset = DatasetDict({
    'train': split_dataset['train'],
    'validation': temp_dataset['train'],
    'test': temp_dataset['test']
})

final_dataset = final_dataset.cast_column("input_ids", Value("large_string"))
final_dataset = final_dataset.cast_column("labels", Value("large_string"))

# # 스키마와 첫 번째 예시 확인
# for split in final_dataset.keys():
#     print(f"[{split} 데이터셋] 스키마:")
#     print(final_dataset[split].features)
#     print(f"[{split} 데이터셋] 첫 번째 예시:")
#     print(final_dataset[split][0])
# 7. 결과 확인
print(final_dataset)

최대 input_ids 길이: 3993
최대 labels 길이: 1000


Casting the dataset: 100%|██████████| 2433/2433 [00:00<00:00, 1358820.46 examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 19463
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 2433
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 2433
    })
})


In [118]:
# # 스키마와 첫 번째 예시 확인
for split in final_dataset.keys():
    print(f"[{split} 데이터셋] 스키마:")
    print(final_dataset[split].features)
    print(f"[{split} 데이터셋] 첫 번째 예시:")
    print(final_dataset[split][0])

[train 데이터셋] 스키마:
{'document': Value(dtype='large_string', id=None), 'summary': Value(dtype='large_string', id=None)}
[train 데이터셋] 첫 번째 예시:
{'document': '[1] 분할보험료가 약정한 시기에 지급되지 아니한 경우 보험자는 상당한 기간을 정하여 보험계약자에게 최고하고 그 기간 안에 보험료가 지급되지 아니한 때에는 그 보험계약을 해지할 수 있으나, 보험계약자와 피보험자가 다른 때에는 상법 제650조 제3항에 따라 피보험자에게도 상당한 기간을 정하여 보험료의 지급을 최고한 뒤가 아니면 그 계약을 해지하지 못한다. [2] 보험계약자 또는 피보험자가 주소변경을 통보하지 아니하는 한 보험증권에 기재된 보험계약자 또는 피보험자의 주소를 보험회사의 의사표시를 수령할 지정장소로 본다는 개인용 자동차보험 특별약관의 규정은 보험회사가 과실 없이 보험계약자 또는 피보험자의 주소 등 소재를 알지 못한 경우에 한하여 적용된다.', 'summary': '약정한 시기에 분할보험료가 지급되지 않았다면, 보험자는 상당한 기간을 정해 보험계약자에게 최고하고 그 기간 안에 보험료가 지급되지 않은 경우 그 보험계약을 해지할 수 있으며, 개인용 자동차보험 특별약관의 규정에 따라 보험계약자 또는 피보험자가 주소변경을 통보하지 않는 것에 한하여 보험증권에 기재된 보험계약자 또는 피보험자의 주소를 보험회사의 의사표시를 수령할 지정장소로 보므로, 보험회사가 과실 없이 보험계약자 또는 피보험자의 소재를 알지 못한 경우에 한하여 적용된다.'}
[validation 데이터셋] 스키마:
{'document': Value(dtype='large_string', id=None), 'summary': Value(dtype='large_string', id=None)}
[validation 데이터셋] 첫 번째 예시:
{'document': '[1] 지입차주와 지입회사 사이의 구체적인 법률관계는 사적자치의

In [158]:
final_dataset['train'][0]

{'input_ids': '[1] 분할보험료가 약정한 시기에 지급되지 아니한 경우 보험자는 상당한 기간을 정하여 보험계약자에게 최고하고 그 기간 안에 보험료가 지급되지 아니한 때에는 그 보험계약을 해지할 수 있으나, 보험계약자와 피보험자가 다른 때에는 상법 제650조 제3항에 따라 피보험자에게도 상당한 기간을 정하여 보험료의 지급을 최고한 뒤가 아니면 그 계약을 해지하지 못한다. [2] 보험계약자 또는 피보험자가 주소변경을 통보하지 아니하는 한 보험증권에 기재된 보험계약자 또는 피보험자의 주소를 보험회사의 의사표시를 수령할 지정장소로 본다는 개인용 자동차보험 특별약관의 규정은 보험회사가 과실 없이 보험계약자 또는 피보험자의 주소 등 소재를 알지 못한 경우에 한하여 적용된다.',
 'labels': '약정한 시기에 분할보험료가 지급되지 않았다면, 보험자는 상당한 기간을 정해 보험계약자에게 최고하고 그 기간 안에 보험료가 지급되지 않은 경우 그 보험계약을 해지할 수 있으며, 개인용 자동차보험 특별약관의 규정에 따라 보험계약자 또는 피보험자가 주소변경을 통보하지 않는 것에 한하여 보험증권에 기재된 보험계약자 또는 피보험자의 주소를 보험회사의 의사표시를 수령할 지정장소로 보므로, 보험회사가 과실 없이 보험계약자 또는 피보험자의 소재를 알지 못한 경우에 한하여 적용된다.'}

### 모델 로드

In [97]:
# BASE_MODEL = "google/gemma-2b-it"
# model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map={"":0}).to(device)
# tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)


In [98]:
# doc = final_dataset['train']['document'][0]
# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)
# messages = [
#     {"role": "user",
#      "content": "...."}
#      ]

# prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)


### Gemma-it 추론

In [99]:
# outputs = pipe(
#     prompt,
#     do_sample=True,
#     temperature=0.2,
#     top_k=50,
#     top_p=0.95,
#     add_special_tokens=True
# )

### 프롬프트 설정 

In [120]:
def generate_prompt(example):
    prompt_list = []
    for i in range(len(example['document'])):
        prompt_list.append(r"""<bos><start_of_turn>user
다음 글을 요약해주세요:

{}<end_of_turn>
<start_of_turn>model
{}<end_of_turn><eos>""".format(example['document'], example['summary']))
    return prompt_list

### 학습-QLoRA


In [121]:
lora_config = LoraConfig(
    r=6,
    lora_alpha = 8,
    lora_dropout = 0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [159]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, PeftModel
from transformers import BitsAndBytesConfig


BASE_MODEL = "google/gemma-2b-it"

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, quantization_config=bnb_config)
model = get_peft_model(model, lora_config) ##
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

`low_cpu_mem_usage` was None, now default to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.60it/s]


In [105]:
# # 스키마와 첫 번째 예시 확인
for split in final_dataset.keys():
    print(f"[{split} 데이터셋] 스키마:")
    print(final_dataset[split].features)
    print(f"[{split} 데이터셋] 첫 번째 예시:")
    print(final_dataset[split][0])
    
	

[train 데이터셋] 스키마:
{'document': Value(dtype='large_string', id=None), 'summary': Value(dtype='large_string', id=None)}
[train 데이터셋] 첫 번째 예시:
{'document': '[1] 분할보험료가 약정한 시기에 지급되지 아니한 경우 보험자는 상당한 기간을 정하여 보험계약자에게 최고하고 그 기간 안에 보험료가 지급되지 아니한 때에는 그 보험계약을 해지할 수 있으나, 보험계약자와 피보험자가 다른 때에는 상법 제650조 제3항에 따라 피보험자에게도 상당한 기간을 정하여 보험료의 지급을 최고한 뒤가 아니면 그 계약을 해지하지 못한다. [2] 보험계약자 또는 피보험자가 주소변경을 통보하지 아니하는 한 보험증권에 기재된 보험계약자 또는 피보험자의 주소를 보험회사의 의사표시를 수령할 지정장소로 본다는 개인용 자동차보험 특별약관의 규정은 보험회사가 과실 없이 보험계약자 또는 피보험자의 주소 등 소재를 알지 못한 경우에 한하여 적용된다.', 'summary': '약정한 시기에 분할보험료가 지급되지 않았다면, 보험자는 상당한 기간을 정해 보험계약자에게 최고하고 그 기간 안에 보험료가 지급되지 않은 경우 그 보험계약을 해지할 수 있으며, 개인용 자동차보험 특별약관의 규정에 따라 보험계약자 또는 피보험자가 주소변경을 통보하지 않는 것에 한하여 보험증권에 기재된 보험계약자 또는 피보험자의 주소를 보험회사의 의사표시를 수령할 지정장소로 보므로, 보험회사가 과실 없이 보험계약자 또는 피보험자의 소재를 알지 못한 경우에 한하여 적용된다.'}
[validation 데이터셋] 스키마:
{'document': Value(dtype='large_string', id=None), 'summary': Value(dtype='large_string', id=None)}
[validation 데이터셋] 첫 번째 예시:
{'document': '[1] 지입차주와 지입회사 사이의 구체적인 법률관계는 사적자치의

In [160]:
print(final_dataset['test'].features)

{'input_ids': Value(dtype='large_string', id=None), 'labels': Value(dtype='large_string', id=None)}


In [161]:
train_data=final_dataset['test']
train_data.features

{'input_ids': Value(dtype='large_string', id=None),
 'labels': Value(dtype='large_string', id=None)}

In [ ]:
# # 데이터셋의 모든 스키마 정보 확인
# print(final_dataset['test'].features)

# # 모든 문자열 필드를 large_string으로 변환할 새 Features 객체 생성
# from datasets import Features, Value

# # 모든 텍스트 필드를 large_string으로 변환
# new_features = {}
# for field_name, field_type in final_dataset['test'].features.items():
#     if field_type.dtype == 'string':
#         new_features[field_name] = Value("large_string")
#     else:
#         new_features[field_name] = field_type

# # 새 스키마로 데이터셋 변환
# train_data = final_dataset['test'].cast(Features(new_features))

{'document': Value(dtype='large_string', id=None), 'summary': Value(dtype='large_string', id=None)}


Casting the dataset: 100%|██████████| 2433/2433 [00:00<00:00, 1246304.55 examples/s]


### 학습

In [162]:
from transformers import Trainer, TrainingArguments
import torch

# ✅ 1. Trainer의 학습 파라미터 설정
training_args = TrainingArguments(
    remove_unused_columns=False,
    output_dir="outputs",
    num_train_epochs=1,
    max_steps=3,
    per_device_train_batch_size=1,
    optim="paged_adamw_8bit",
    warmup_steps=10,
    learning_rate=2e-5,
    fp16=True,
    logging_steps=10,
    push_to_hub=False,
    report_to='none',
)

In [166]:
from transformers import Trainer
from datasets import Dataset, DatasetDict

if not isinstance(train_data, Dataset):
    train_data = Dataset.from_dict(train_data)
    
trainer = Trainer(
    model=model.to("cuda:0"),
    args=training_args,
    train_dataset=train_data,
)

trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


TypeError: can only join an iterable

In [114]:
# train_data = final_dataset['test']
# train_data = train_data.cast(Features({
#     "document": Value("large_string"),
#     "summary": Value("large_string")
# }))
trainer = SFTTrainer(
    model=model.to(device),
    train_dataset=train_data,
#    max_seq_length=512,
    args=TrainingArguments(
        output_dir="outputs",
        num_train_epochs = 1,
        max_steps=3,
        per_device_train_batch_size=1,
#        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        warmup_steps=10,
        learning_rate=2e-5,
        fp16=True,
        logging_steps=10,
        push_to_hub=False,
        report_to='none',
        
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)



Converting train dataset to ChatML:  82%|████████▏ | 1999/2433 [00:01<00:00, 1936.53 examples/s]


ArrowInvalid: offset overflow while concatenating arrays, consider casting input from `string` to `large_string` first.

In [92]:
trainer.train()

RuntimeError: Caught RuntimeError in replica 1 on device 1.
Original Traceback (most recent call last):
  File "/home/wanted-1/miniconda3/envs/whisper/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 96, in _worker
    output = module(*input, **kwargs)
  File "/home/wanted-1/miniconda3/envs/whisper/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1739, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/wanted-1/miniconda3/envs/whisper/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1750, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/wanted-1/miniconda3/envs/whisper/lib/python3.10/site-packages/peft/peft_model.py", line 1719, in forward
    return self.base_model(
  File "/home/wanted-1/miniconda3/envs/whisper/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1739, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/wanted-1/miniconda3/envs/whi

In [ ]:
ADAPTER_MODEL = "lora_adapter"

trainer.model.save_pretrained(ADAPTER_MODEL)

In [ ]:
# !ls -alh lora_adapter

### Q-LoRA 와 모델 합치기 

In [ ]:
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, torch_dtype=torch.float16)

model = model.merge_and_unload()
model.save_pretrained('gemma-2b-it-sum-ko')

# !ls -alh ./gemma-2b-it-sum-ko

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.41it/s]


NameError: name 'ADAPTER_MODEL' is not defined

### Fine-tuned 모델 로드 

In [ ]:
BASE_MODEL = "google/gemma-2b-it"
FINETUNE_MODEL = "./gemma-2b-it-sum-ko"

finetune_model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)

In [ ]:
pipe_finetuned = pipeline("text-generation", model=finetune_model, tokenizer=tokenizer, max_new_tokens=512)
doc = dataset['test']['document'][10]

messages = [
    {
        "role": "user",
        "content": "다음 글을 요약해주세요:\n\n{}".format(doc)
    }
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)


In [ ]:
outputs = pipe_finetuned(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)
print(outputs[0]["generated_text"][len(prompt):])